In [5]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import os
from PIL import Image
%matplotlib inline

In [7]:
def trainPics(path):
    path = os.getcwd() + path 
    
    trainFile= open("trainData.csv","w")
    skinCount = np.zeros((256, 256, 256))
    nonSkinCount = np.zeros((256, 256, 256))
    
    
    
    for root, dirs, files in os.walk(path + 'Mask\\'):
        for filename in files:
            #print(filename)
            if filename.endswith(".bmp") : 
                im = Image.open(path + 'Mask\\'+ filename, 'r')
                filename, extension = filename.split('.')
                imOriginal = Image.open(path + filename+'.jpg', 'r') 
                pix_val_mask = list(im.getdata())
                pix_val = list(imOriginal.getdata())
                pixelCount = len(pix_val)
                
                for i in range(pixelCount) :
                    if pix_val_mask[i][0]>=230 and pix_val_mask[i][1]>=230 and pix_val_mask[i][2] >=230 :
                        nonSkinCount[pix_val[i][0]][pix_val[i][1]][pix_val[i][2]] += 1 
                    else:
                        skinCount[pix_val[i][0]][pix_val[i][1]][pix_val[i][2]] += 1 
                           
            else:
                continue
    trainFile.write("red,green,blue,likelihoodRatio\n")           
    for i in range(256):
        for j in range(256):
            for k in range(256):
                if nonSkinCount[i][j][k] == 0:
                    likelihoodRatio = 0
                else:
                    likelihoodRatio = skinCount[i][j][k]/nonSkinCount[i][j][k]
                trainFile.write(str(i) +','+ str(j) +','+ str(k) +','+ str(likelihoodRatio) + '\n')
    
    trainFile.close()
        

path = '\ibtd\\'
trainPics(path)

In [8]:
def skin_detection(image_path, train_path):
    testData = Image.open(image_path)
    height, width = testData.size

    pixArr = np.zeros((height, width, 3) , dtype= np.uint8)
    rgbProb = np.zeros((256, 256, 256))
    
    
    model = pd.read_csv(train_path)
#     for index, row in model.iterrows():
#         r = int(row['red'])
#         g= int(row['green'])
#         b = int(row['blue'])
#         #print(r, g, b)
#         rgbProb[r][g][b] = row['likelihoodRatio']
    
    
    
    for i in range(height):
        for j in range(width):
            r, g, b = testData.getpixel((i, j))
            #print(model[ (model['red']==r) & (model['green']==g) & (model['blue']==b) ].reset_index()['likelihoodRatio'][0])
            #if  model[ (model['red']==r) & (model['green']==g) & (model['blue']==b) ].reset_index()['likelihoodRatio'][0] <= 0.3 :
            if model['likelihoodRatio'][256*256*r+ 256*g +b] <=0.3 :
                pixArr[i][j] = [255, 255, 255]
            else:
                pixArr[i][j] = [r, g, b]
    
    im2 = Image.fromarray(pixArr)
    im2.save('maskedTest.jpg')
    
image_path = 'test1.jpg'
train_path = 'trainData.csv'
skin_detection(image_path, train_path)


In [4]:
model = pd.read_csv('trainData.csv')
model.head()

,red,green,blue,likelihoodRatio
0,0,0,0,0.0
1,0,0,1,1.0
2,0,0,2,0.0
3,0,0,3,1.0
4,0,0,4,0.0
